In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
from IPython.display import display, Markdown
from itables import init_notebook_mode
import os
import seaborn as sns
import time
import json
import ast
import dask.dataframe as dd
from collections import Counter

init_notebook_mode(all_interactive=True)

In [ ]:

def process_json_to_csv_by_folder():
    base_dir = os.path.join(os.path.dirname(os.getcwd()), 'archive_data')
    
    start_time = time.time()

    for subdir, _, files in os.walk(base_dir):
        df_list = []

        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(subdir, file)
                
                try:
                    df = pd.read_json(file_path)
                    df_list.append(df)
                except ValueError as ve:
                    print(f"Błąd podczas przetwarzania pliku {file_path}: {ve}")

        if df_list:
            combined_df = pd.concat(df_list, ignore_index=True)
            folder_name = os.path.basename(subdir)
            csv_file_name = folder_name + '.csv'
            csv_file_path = os.path.join(base_dir, csv_file_name)
            combined_df.to_csv(csv_file_path, index=False)
            print(f"Zapisano plik CSV.")

    end_time = time.time()
    print(f"Czas przetwarzania: {end_time - start_time} sekund")

# Uruchom przetwarzanie danych z JSON do CSV
process_json_to_csv_by_folder()

In [ ]:
# Funkcja wczytywania danych do df
def combine_csv_to_df():
    base_dir = os.path.join(os.path.dirname(os.getcwd()), 'archive_data')
    combined_df_list = []

    for file in os.listdir(base_dir):
        if file.endswith('.csv'):
            csv_file_path = os.path.join(base_dir, file)
            print(f"Wczytywanie pliku CSV.")

            try:
                df = pd.read_csv(csv_file_path)
                combined_df_list.append(df)
            except pd.errors.EmptyDataError as e:
                print(f"Błąd podczas wczytywania pliku CSV {csv_file_path}: {e}")

    if not combined_df_list:
        print("Żadne pliki CSV nie zostały wczytane.")
        return pd.DataFrame()

    combined_df = pd.concat(combined_df_list, ignore_index=True)
    return combined_df

# Uruchom łączenie plików CSV w jeden DataFrame
df = combine_csv_to_df()


# 1. Ogólny przegląd danych. 

In [ ]:
df.shape

In [ ]:
df.head(50)


In [ ]:
df.columns


In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
# Początkowy przegląd losowych 5 wierszy
df.sample(5)

In [ ]:
# Sprawdzenie unikalnych wartości w kolumnach
df.nunique()

In [ ]:

mark1= f"""
## Obserwacje wstępne:
### 1. Dane archiwalne ofert pracy ze strony JustJoinIT.
### 2. Dane zawierają **{len(df)}** wierszy oraz **{len(df.columns)}** kolumn.
"""
display(Markdown(mark1))

# 2. Przygotowanie danych

In [ ]:
# Usuniecie nie optrzebnych kolumn
df = df[['title', 'street', 'city', 'country_code', 
       #'address_text',
       'marker_icon', 'workplace_type', 'company_name', 
       #'company_url', 'company_size', 
       'experience_level', 
       #'latitude', 'longitude',
       'published_at', 'remote_interview', 'id', 'employment_types',
       #'company_logo_url', 
       'skills', 'remote', 'open_to_hire_ukrainians',
       #'display_offer', 
       #'multilocation', 'way_of_apply'
       ]].copy()

In [ ]:
df.dtypes

In [ ]:

df['published_at'] = pd.to_datetime(df['published_at'], format='ISO8601', utc=True)

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df.loc[df.duplicated(subset=['title'])]

In [ ]:
df.query('title == "Analityk Systemowy (Integracje Systemów)"')

In [ ]:
df = df.loc[~df.duplicated(subset=['title', 'street', 'city', 'company_name', 'published_at'])].reset_index(drop=True).copy()

In [ ]:
df.shape

### Przeprowadzone operacje wstępne:
#### 1. Sprawdzenie jakie dane zawiera DataFrame oraz usunięcie zbędnych kolumn.
#### 2. Naprawienie kolumny z czasem publikacji.
#### 3. Wykrycie dużej ilości duplikatów, usunięcie powtarzających się ofert pracy w danych.


<br><br>


# 3. Przegląd pojedyńczysz kolumn
<br><br>

### 1. Stanowiska pracy i TOP15 poszukiwanych pracowników.

In [ ]:
# Sprawdzenie ilości unikatowych stanowisk pracy
df['title'].nunique()

In [ ]:
# Sprawdzenie powtarzania się stanowisk pracy
df['title'].value_counts()

In [ ]:

ax = df['title'].value_counts() \
    .head(15) \
    .plot(kind='bar', title='Top 15 ofert pracy')
ax.set_xlabel('Popularne zawody')
ax.set_ylabel('Ilości')


In [ ]:

top_15_titles = df['title'].value_counts().head(15)
top_15_text = '\n'.join([f"- {title}: {count}" for title, count in top_15_titles.items()])

mark2= f"""
### Obserwacje:
#### 1. Dane zawierają: **{df['title'].nunique()}** stanowisk pracy z: **{df['company_name'].nunique()}** firm.
#### 2. Pierwsza piętnastka najczęściej wystepujących ofert:\n{top_15_text}.
"""
display(Markdown(mark2))

### 2. Typy pracy, poziom doświadczenia oraz rekrutacja zdalna.

In [ ]:
# Sprawdzenie ilości unikatowych typów pracy
df['workplace_type'].value_counts()

In [ ]:
# Sprawdzenie ilości unikalnych poziomów doświadczenia
df['experience_level'].value_counts() 

In [ ]:
# Sprawdzenie liczba ofert pracy z możliwościa rekrutacji zdalnej
df['remote'].value_counts() 

In [ ]:

worktyp = df['workplace_type'].value_counts()
total_worktyp = worktyp.sum() 
worktyp_counts = '\n'.join([f"- {worktyp}: {count} ({(count / total_worktyp * 100):.2f}%)" for worktyp, count in worktyp.items()])

expeLvl = df['experience_level'].value_counts()
total_expeLvl = expeLvl.sum()
expeLvl_counts = '\n'.join([f"- {expeLvl}: {count} ({(count / total_expeLvl * 100):.2f}%)" for expeLvl, count in expeLvl.items()])

remoteInt = df['remote'].value_counts()
total_remoteInt = expeLvl.sum()
remoteInt_counts = '\n'.join([f"- {remoteInt}: {count} ({(count / total_remoteInt * 100):.2f}%)" for remoteInt, count in remoteInt.items()])

In [ ]:
#Wykresy 
worktyp_labels = worktyp.index
worktyp_sizes = worktyp.values

expeLvl_labels = expeLvl.index
expeLvl_sizes = expeLvl.values

remoteInt_labels = remoteInt.index
remoteInt_sizes =  remoteInt.values

# Tworzenie wykresów kołowych
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Wykres dla workplaceType
axes[0].pie(worktyp_sizes, labels=worktyp_labels, autopct='%1.1f%%', colors=plt.cm.Paired.colors)
axes[0].set_title('Rodzaje pracy')

# Wykres dla experienceLevel
axes[1].pie(expeLvl_sizes, labels=expeLvl_labels, autopct='%1.1f%%', colors=plt.cm.Accent.colors)
axes[1].set_title('Poziomy doświadczenia')

# Wykres dla remoteInterview
axes[2].pie(remoteInt_sizes, labels=remoteInt_labels, autopct='%1.1f%%', colors=plt.cm.Set1.colors)
axes[2].set_title('Rekrutacja zdalna')

plt.tight_layout()
plt.show()

In [ ]:

mark3= f"""
### Wnioski:
#### 1. Dane zawierają: **{df['workplace_type'].nunique()}** rodzaje pracy:\n {worktyp_counts}.  
#### 2. Liczba ofert pracy dla poszczególnych poziomów doświadczenia:\n{expeLvl_counts}.
#### 3. Liczba ofert pracy z możliwościa rekrutacji zdalnej:\n{remoteInt_counts}.
"""
display(Markdown(mark3))

### 3. Sprawdzenie częstotliwości ofert pracy dla Ukraińców.

In [ ]:

forUA = df['open_to_hire_ukrainians'].value_counts()
total_forUA = forUA.sum() 
forUA_counts = '\n'.join([f"- {forUA}: {count} ({(count / total_forUA * 100):.2f}%)" for forUA, count in forUA.items()])


In [ ]:
# Generowanie wykresu kołowego
plt.pie(forUA, labels=forUA.index, autopct='%1.1f%%', startangle=90)
plt.title('Dostępne oferty dla Ukraińców')
plt.show()

In [ ]:

mark4= f"""
### Obserwacje:
#### Oferty dla Ukraińców: \n {forUA_counts}. 
"""
display(Markdown(mark4))

<br><br>

### 4. Analiza ofert pracy w poszczególnych miastach.

In [ ]:
# Sprawdzenie ilości unikatowych miejsc zatrudnienia
df['city'].nunique()

In [ ]:

# Ocena liczby unikatowych lokalizacji zatrudnienia
unique_locations_initial = df['city'].nunique()

city_mapping = {
    'Poland (Remote)' : 'Polska',
    'Poland' : 'Polska', 
    'Remote Poland' : 'Polska',
    'Warsaw' : 'Warszawa',
    'Warszawa (Centrum)' : 'Warszawa',
    }

# Normalizacja nazw miast
df['city'] = df['city'].replace(city_mapping)

# Wyliczenie procentowego udziału każdej lokalizacji
location_share = df['city'].value_counts(normalize=True) * 100

# Przypisanie lokalizacji do kategorii "Inne", jeśli występują mniej niż w 3,5% przypadków
total_small_share = location_share[location_share < 3.5].sum()

# Redukcja do lokalizacji występujących co najmniej w 3,5% przypadków
location_share = location_share[location_share >= 3.5]

# Dodanie kategorii "Inne" do zsumowanych pomniejszych kategorii
location_share['Inne'] = total_small_share

# Tworzenie wizualizacji w formie wykresu pi
plt.pie(location_share, labels=location_share.index, autopct='%1.1f%%', startangle=270)
plt.title('Oferty zatrudnienia w poszczególnych miastach')
plt.show()

# Mapowanie lokalizacji na podstawie ich częstości występowania
location_count = df['city'].value_counts()

# Stworzenie indeksu bazującego na częstości wystąpień
location_index_map = {loc: idx + 1 for idx, loc in enumerate(location_count.index)}

# Aktualizacja kolumny 'city' przy użyciu indeksu bez wpływu na wykres
df['city_index'] = df['city'].map(location_index_map)

In [ ]:

total_citytyp = location_count.sum() 
# Wyliczanie procentowego udziału dla każdej lokalizacji
location_percentage = (location_count / total_citytyp) * 100

# Filtrowanie lokalizacji do wyświetlenia
significant_locations = {location: count for location, count in location_count.items() if location_percentage[location] >= 2}

significant_citytyp_counts = '\n'.join([
    f"- {location}: {count} ({(count / total_citytyp * 100):.2f}%)"
    for location, count in significant_locations.items()
])

mark5 = f"""
### Obserwacje:
#### Prezentacja ilości ofert pracy w danych miejscowościach i ich procent w danych:\n{significant_citytyp_counts}.
#### W danych historycznych możemy zaobserwować że największe zapotrzebowanie na specjalistów w branży IT jest w miejscowościach:
#### - Warszawa
#### - Wrocław
#### - Kraków
"""

display(Markdown(mark5))

<br><br>

### 5. Analiza czasu publikacji ogłoszeń.

In [ ]:
# Konwersja kolumny 'publishedAt' na datę i czas
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')

# Wyodrębnienie godziny z kolumny 'publishedAt'
df['publication_hour'] = df['published_at'].dt.hour

# Zliczanie liczby ogłoszeń na każdej godzinie
hourly_counts = df['publication_hour'].value_counts().sort_index()

# Tworzenie wykresu słupkowego
plt.figure(figsize=(12, 6))
plt.bar(hourly_counts.index, hourly_counts.values, color='skyblue')
plt.xlabel('Godzina publikacji')
plt.ylabel('Liczba ogłoszeń')
plt.title('Liczba ogłoszeń według godzin publikacji')
plt.xticks(range(24))  
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

#### Obserwacje:
* ##### Dane historyczne zamieszczane były w godzinach pracy miedzy 7 a 16.
* ##### Ta obserwacja prowadzi nas do pytania, czy oferty są sprawdzane przez administratorów/moderatorów i dodawane w ich godzinach pracy?
* ##### W godzinach wieczornych między 17-6 rano widzimy minimalną aktywność publikacji ofert, co może nam sugerować dodawanie ofert np. zagranicznych.

<br> 

### 6. Analiza wymaganych umiejętności oraz sposób zatrudnienia.
<br><br>

#### 1. Umiejętności

In [ ]:

skill_df = df[['title', 'skills']]
skill_df

In [ ]:

# Funkcja do wyodrębnienia nazw umiejętności
def extract_skill_names(skills_str):
    try:
        skills = ast.literal_eval(skills_str)
        return [skill['name'] for skill in skills if 'name' in skill]
    except (ValueError, SyntaxError):
        return []

# Wyodrębnienie i zliczenie umiejętności
all_skills = skill_df['skills'].apply(extract_skill_names)
all_skills_flat = [skill for sublist in all_skills for skill in sublist]

# Zliczanie i wybieranie top 30 umiejętności
skill_counts = Counter(all_skills_flat)
top_skills = skill_counts.most_common(30)

# Przygotowanie danych do wykresu
names, counts = zip(*top_skills)

# Tworzenie wykresu popularności umiejętności
plt.figure(figsize=(12, 8))
plt.bar(names, counts, color='skyblue')
plt.title('Top 30 najczęściej występujących umiejętności')
plt.xlabel('Umiejętności')
plt.ylabel('Liczba wystąpień')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:

# Wyliczenie ilości unikalnych umiejętności
unique_skills_count = len(skill_counts)

# Całkowita liczba wystąpień wszystkich umiejętności
total_occurrences = sum(skill_counts.values())

# Przygotowanie tekstu dla top 30 umiejętności
top_30_text = '\n'.join([
    f"- {skill}: {count} ({count / total_occurrences:.2%})"
    for skill, count in top_skills
])

# Tworzenie markdownu z podsumowaniem
mark6 = f"""
### Obserwacje:
#### 1. Ilość unikalnych umiejętności w danych: {unique_skills_count}
#### 2. Trzydzieści najczęściej wymienianych umiejętności w ofertach pracy:\n{top_30_text}
"""

display(Markdown(mark6))

#### 2. Sposób zatrudnienia / wspólpracy


In [ ]:

# Funkcja do wyodrębnienia typów współpracy
def extract_collaboration_types(types_str):
    try:
        types = ast.literal_eval(types_str)
        return [item['type'] for item in types if 'type' in item]
    except (ValueError, SyntaxError):
        return []

# Wyodrębnienie i zliczenie typów współpracy
all_collaboration_types = df['employment_types'].apply(extract_collaboration_types)
all_collaboration_flat = [t for sublist in all_collaboration_types for t in sublist]

# Liczenie wystąpień każdego typu współpracy
collaboration_counts = Counter(all_collaboration_flat)
collaboration_counts = pd.Series(collaboration_counts).sort_values(ascending=False)

# Obliczanie procentowych wartości
total_counts = collaboration_counts.sum()
collaboration_percents = (collaboration_counts / total_counts * 100).round(1)

# Przygotowanie tekstu do podsumowania
collaboration_text = '\n'.join(
    [f"- {method}: {count} ({percent}%)" for method, count, percent in zip(collaboration_counts.index, collaboration_counts.values, collaboration_percents.values)]
)

# Wyświetlanie wykresu kołowego
sizes = collaboration_counts.values
labels = collaboration_counts.index
colors = plt.cm.tab20.colors[:len(labels)]  # Wybieranie kolorów z palety

plt.figure(figsize=(8, 6))  # Ustalenie rozmiaru wykresu

wedges, texts, autotexts = plt.pie(
    sizes, 
    autopct='%1.1f%%', 
    startangle=90, 
    colors=colors,
    textprops=dict(color="white")  # Tekst wartości procentowej na białym tle
)

for autotext in autotexts:
    autotext.set_fontsize(10)
    autotext.set_color('white')

plt.title('Typy Współpracy')
plt.axis('equal')  # Zapewnienie, że koło jest okrągłe

plt.legend(wedges, labels, title="Typy współpracy", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.show()

In [ ]:

# Tworzenie markdownowych wniosków do wyświetlenia
mark7= f"""
### Obserwacje dotyczące sposobów współpracy:
#### Najczęściej wymieniane sposoby współpracy:\n{collaboration_text}.
"""

# Wyświetlanie wyników
display(Markdown(mark7))

In [ ]:
df

### 7. Analiza wynagrodzeń

In [ ]:

# Funkcja do wyodrębnienia danych o wynagrodzeniach
def extract_salary_info(types_str):
    try:
        types = ast.literal_eval(types_str)
        salary_data = []
        for item in types:
            salary = item.get('salary')
            if salary:
                salary_from = salary.get('from')
                salary_to = salary.get('to')
                salary_currency = salary.get('currency')
                if salary_from is not None and salary_to is not None:
                    salary_data.append({
                        'type': item.get('type'),
                        'salary': int(salary_from),
                        'range': 'from',
                        'currency': salary_currency
                    })
                    salary_data.append({
                        'type': item.get('type'),
                        'salary': int(salary_to),
                        'range': 'to',
                        'currency': salary_currency
                    })
        return salary_data
    except (ValueError, SyntaxError):
        return []

# Rozpakowywanie danych o wynagrodzeniach.
salary_info_list = df['employment_types'].apply(extract_salary_info)

# Konwersja do płaskiej listy słowników
salary_info_flat = [item for sublist in salary_info_list for item in sublist]

# Konwersja do DataFrame
salary_df = pd.DataFrame(salary_info_flat)




In [ ]:
salary_df

In [ ]:

# Definicje kursów wymiany do PLN
exchange_rates = {
    'usd': 3.97,
    'eur': 4.16,
    'gbp': 4.99,
    'chf': 4.40
}

def convert_to_pln(row):
    currency = row['currency']

    if currency in exchange_rates:
        rate = exchange_rates[currency]
        # Konwersja do float, przeliczenie, zaokrąglenie i konwersja do int
        row['salary'] = int(round(float(row['salary']) * rate))
        row['currency'] = 'pln'
    else:
        # Gdy waluta już jest PLN, zaokrąglamy i konwertujemy do int
        row['salary'] = int(round(float(row['salary'])))
    return row

# Iteracyjne stosowanie konwersji dla każdej linii tabeli salary_df
salary_df = salary_df.apply(convert_to_pln, axis=1)

In [ ]:
# Warunkowe dzielenie/mnożenie dla wynagrodzenia
salary_df['salary'] = salary_df['salary'].apply(lambda x: x / 12 if x > 80000 else x)
salary_df['salary'] = salary_df['salary'].apply(lambda x: x * 160 if x < 500 else x)


In [ ]:
salary_df.describe()

In [ ]:
salary_df.plot(kind='scatter', 
               x='salary',
               y='type',
               )
plt.show()

In [ ]:
# Tworzenie wykresu boxplot dla wynagrodzeń
plt.figure(figsize=(12, 6))
sns.boxplot(x='type', y='salary', hue='range', data=salary_df, palette='pastel', width=0.4)
plt.title('Średnie wynagrodzeia dla typów współpracy')
plt.xlabel('Typ współpracy')
plt.ylabel('Wynagrodzenie (PLN)')
plt.yticks(ticks=range(0, int(salary_df['salary'].max()) + 5000, 5000))
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
sns.pairplot(salary_df, 
            vars=['type', 'salary', 'currency'])
plt.show()

In [ ]:

# Generowanie tekstu markdown
markdown_text = f"""
Obserwacje wynagrodzeń we wszystkich typach współpracy:

1. Po wstępnych obserwacjach wynagrodzeń w ofertach pracy znaleziono wynagrodznia w obcych walutach.
   - wszystkie waluty zagraniczne przekonwertowane na pln dla lepszego zrozumienia danych.
2. Obserwacje minimalnego i maksymalnego wynagrodzenia - dokonano naprawy wartości w danych.
   - przytuszczalnie najmniejsze wynagrodzenia odnosiły się do stawek dziennych przez co zostały podniesione do wartości miesięcznej.
   - wartości max wynagrodzeń zostały przypuszczalnie zinterpretowane na wynagrodzeń rocznych i zostały poprawione na wynagrodzenia miesięczne.
3. Obeserwacje po naprawie danych pokazują przedziały płacowe na bardzo zróżnicowanym poziomie.
   W typach współpracy 'b2b' oraz 'permanent' znacząca ilość wartości odstających przekraczających maxymalne średnie wynagrodzenie.

"""

print(markdown_text)

# 4. Analiza trendów

In [ ]:
df

In [ ]:
# Filtracja danych według zakresów dat
df_22_01_09 = df[(df['published_at'] >= '2022-01-01') & (df['published_at'] <= '2022-09-30')]
df_23_01_09 = df[(df['published_at'] >= '2023-01-01') & (df['published_at'] <= '2023-09-30')]
df_21_10_12 = df[(df['published_at'] >= '2021-10-01') & (df['published_at'] <= '2021-12-31')]
df_22_10_12 = df[(df['published_at'] >= '2022-10-01') & (df['published_at'] <= '2022-12-31')]

In [ ]:

count_21_10_12 = len(df_21_10_12)
count_22_10_12 = len(df_22_10_12)
count_22_01_09 = len(df_22_01_09)
count_23_01_09 = len(df_23_01_09)

percent_change_21_22_q4 = ((count_22_10_12 - count_21_10_12) / count_21_10_12 * 100) if count_21_10_12 != 0 else 0
percent_change_22_23_q1_q3 = ((count_23_01_09 - count_22_01_09) / count_22_01_09 * 100) if count_22_01_09 != 0 else 0


In [ ]:

mar = f"""
<h2>1) Analiza archiwalnych ofert pracy:</h2> 

<div style='font-size: 20px'>

* W okresie od Października do Grudnia 2021 roku opublikowano: **{count_21_10_12}** ofert pracy.
* W okresie od Października do Grudnia 2022 roku opublikowano: **{count_22_10_12}** ofert pracy.
* Porównując dane z roku 2022 a z 2021 widać wzrost publikacji ogłoszeń aż o: **{percent_change_21_22_q4:.2f}%**,\n
    przez co można stwierdzić rozwój branży IT i zapotrzebowanie na nowych specjalistów.
<br><br>

* W okresie od Stycznia do Września 2022 roku opublikowano: **{count_22_01_09}** ofert pracy.
* W okresie od Stycznia do Września 2023 roku opublikowano: **{count_23_01_09}** ofert pracy.
* Co stanowi: **{percent_change_22_23_q1_q3:.2f}%** wzrostu ofert pracy w danych okresach,\n 
    przez co można stwierdzić że zapotrzebowanie na wykwalifikowanych pracowników z roku na rok wzrasta.
</div>
"""

display(Markdown(mar))

In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
ax1 = df_22_01_09['title'].value_counts() \
    .head(5) \
    .plot(ax=axes[0], 
          kind='bar', 
          title='Top 5 ofert pracy w okresie 01-09.2022')
ax1.set_xlabel('Popularne zawody')
ax1.set_ylabel('Ilości')

ax2 = df_23_01_09['title'].value_counts() \
    .head(5) \
    .plot(ax=axes[1], 
          kind='bar', 
          title='Top 5 ofert pracy w okresie 01-09.2023')
ax2.set_xlabel('Popularne zawody')
ax2.set_ylabel('Ilości')

plt.tight_layout()
plt.show()


In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))
ax1 = df_21_10_12['title'].value_counts() \
    .head(5) \
    .plot(ax=axes[0], 
          kind='bar', 
          title='Top 5 ofert pracy w okresie 10-12.2021')
ax1.set_xlabel('Popularne zawody')
ax1.set_ylabel('Ilości')

ax2 = df_22_10_12['title'].value_counts() \
    .head(5) \
    .plot(ax=axes[1],
          kind='bar', 
          title='Top 5 ofert pracy w okresie 10-12.2022')
ax2.set_xlabel('Popularne zawody')
ax2.set_ylabel('Ilości')

plt.tight_layout()
plt.show()

### Obserwacje wstępne:
#### Porównując stanowiska pracy z ostatnich lat w danych okresach czasu swierdzono że:
####   * Największe zapotrzebowanie było na stanowisko: **Java Developer**, **DevOps Enginner** oraz **PHP Developer**.


In [ ]:

explvl1 = df_21_10_12['experience_level'].value_counts()
total_explvl1 = explvl1.sum()
expeLvl1_counts = '\n'.join([f"- {explvl1}: {count} ({(count / total_explvl1 * 100):.2f}%)" for explvl1, count in explvl1.items()])
explvl1_labels = explvl1.index
explvl1_sizes = explvl1.values

explvl2 = df_22_10_12['experience_level'].value_counts()
total_explvl2 = explvl2.sum()
expeLvl2_counts = '\n'.join([f"- {explvl2}: {count} ({(count / total_explvl2 * 100):.2f}%)" for explvl2, count in explvl2.items()])
explvl2_labels = explvl2.index
explvl2_sizes = explvl2.values

explvl3 = df_22_01_09['experience_level'].value_counts()
total_explvl3 = explvl3.sum()
expeLvl3_counts = '\n'.join([f"- {explvl3}: {count} ({(count / total_explvl3 * 100):.2f}%)" for explvl3, count in explvl3.items()])
explvl3_labels = explvl3.index
explvl3_sizes = explvl3.values

explvl4 = df_23_01_09['experience_level'].value_counts()
total_explvl4 = explvl3.sum()
expeLvl4_counts = '\n'.join([f"- {explvl4}: {count} ({(count / total_explvl4 * 100):.2f}%)" for explvl4, count in explvl4.items()])
explvl4_labels = explvl4.index
explvl4_sizes = explvl4.values

In [ ]:
# Tworzenie wykresów kołowych
fig, axes = plt.subplots(1, 4, figsize=(18, 6))

axes[0].pie(explvl1_sizes, labels=explvl1_labels, autopct='%1.1f%%', colors=plt.cm.Accent.colors)
axes[0].set_title('Poziom doświadczenia *10-12.2021*')

axes[1].pie(explvl2_sizes, labels=explvl2_labels, autopct='%1.1f%%', colors=plt.cm.Accent.colors)
axes[1].set_title('Poziom doświadczenia *10-12.2022*')

axes[2].pie(explvl3_sizes, labels=explvl3_labels, autopct='%1.1f%%', colors=plt.cm.Accent.colors)
axes[2].set_title('Poziom doświadczenia *01-09.2022*')

axes[3].pie(explvl4_sizes, labels=explvl4_labels, autopct='%1.1f%%', colors=plt.cm.Accent.colors)
axes[3].set_title('Poziom doświadczenia *01-09.2023*')
plt.tight_layout()
plt.show()



### Analiza archiwalnych ofert pracy z podziałem na poziomy doświadczenia: 

* #### Z danych archiwalnych wynika że coraz więcej firm wymaga doświadczenia na poziomie *Senior*. 
* #### W danych jest ponad połowa ofert pracy z wymaganym doświadczeniem na poziomie *Mid* .
* #### Niepokojącym trendem spadkowym możemy określić znaczne zmniejszenie zapotrzebowania na osoby z doświadczeniem na poziomie *Junior*.


<br><br><br>

## Podsumowanie Analizy Danych


### 1. Świadomość Danych
- #### Źródła: Archiwalne dane ofert pracy z JostJoinIT 
- #### Wielkość:
  - #### Rekordy: 2096717
  - #### Kolumny: 15

### 2. Podstawowe Statystyki
- #### Typy danych: bool, float, int, object.

### 3. Jakość Danych
- #### Brakujące wartości: Kolumny tekstowe(object) nie posiadają bakujących wartości,
  #### w kolumnach z płacami wystepują brakujące wartości.
- #### Zidentyfikowano błedy w kolumnach z wynagrodzeniami, ceny podawane w różnych walutach,
  #### prawdopodobnie w stawkach dniowych jak i rocznych zostały odpowiednio przygotowane do analizy.
- #### Duplikaty: Ponad połowa danych była duplikatami które zostały usunięte przed analizą.

### 4. Podsumowania i Wnioski
- #### Główne wnioski:
  - #### Dane poddane analizie zawierają : 33818 stanowisk pracy z 7676 firm.
    - #### Trzy najczęściej wystepujące stanowiska:
      - #### Java Developer
      - #### DevOps Enginer
      - #### PHP Developer
  - #### Z danych wynika że prawie wszystkie oferty pracy były zamieszczane w godzinach pracy: 7 - 16,
    #### reszta ofert była opublikowana w godzinach nocnych: 17 - 6, co może sugerować dodawanie ofert np. zagranicznych.
  - #### Dane przedstawiają trzy sposoby wykonywania pracy:
    - #### Praca zdalna: 74.93% 
    - #### Praca hybrydowa: 22.38%
    - #### Praca biurowa: 2.69%
      - #### Biorąc pod uwagę okres który obejmuje wszystkie ogłoszenia można stwierdzić że najmniejsze zainteresowanie pracownikami biurowymi było spowodowane przez COVID-19.
  - #### Najwiecej ofert pracy jest w:
    - #### Warszawa 20.22%
    - #### Wrocław 11.52%
    - #### Kraków 11.44%
  - #### Aż 74.92% ofert pracy umożliwia rekrutacje zdalną, a 35.26% z ofert umożliwia aplikowanie obcokrajowcom z Ukrainy.
  - #### Stwierdzono wzrost ilości ofert pracy, analizując dane z lat 2022-2023.
  - #### Stanowiska z wymaganym doświadczeniem na poziomie *Mid* cieszą się największą popularnościa bo zajmują ponad 50% dostepnych ofert.
  - #### Przybywa ofert pracy z doświadczeniem na poziomie *Senior*.
  - #### Bardzo niepokojącym odkryciem jest że ilość ofert pracy z początkującym doświadczeniem na poziomie *Junior*, 
    #### których w zbiorze danych jest zaledwie 6.3%. Po wykonaniu głębszej analizy zauważono spadek tych stanowisk o około 40%. 
  - #### Przedziały płacowe sa na bardzo zróżnicowanym posiomie.
    - #### W typach współpracy b2b oraz permanent znacząca ilość wartości odstających przekracza maksymalne średnie wynagrodzenie.

- ### Następne kroki: 
  - #### Zaleca się wykonanie porównania archiwalnych danych z nowymi w zakresie spadkowego trendu ofert pracy na rynku *Juniorskim*. 
  - #### Zaleca się sprawdzenie trendów dla poziomów doświadczeń: *Senior* oraz *Mid*.



In [8]:
!jupyter nbconvert archive_eda_jjit.ipynb --to html --no-input --no-prompt --output archive_eda_jjit.html

[NbConvertApp] Converting notebook archive_eda_jjit.ipynb to html
[NbConvertApp] Writing 298611 bytes to archive_eda_jjit.html
